In [44]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma, bernoulli
from statsmodels.tsa.stattools import acf
import datetime
import sys
import os

from tqdm.notebook import tqdm
from codebase.plot import * 
from codebase.data import * 
from codebase.file_utils import save_obj, load_obj

from modelresultsbinary_4chains import *
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
log_dir = "./log/FND/PPP/20200413_195033_FND_s1m6//"
data = load_obj('data', log_dir)
ps = load_obj('ps', log_dir)
ps.keys()


odict_keys(['alpha', 'yy', 'beta', 'Marg_cov', 'Omega_cov'])

In [46]:
data

{'D': array([[1, 0, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 1, 1],
        ...,
        [1, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 1],
        [1, 0, 1, 1, 0, 0]]), 'N': 565, 'J': 6, 'K': 2}

In [47]:
print(open(log_dir+'model.txt').read())

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  int<lower=0, upper=1> DD[N, J];
}

transformed data{
  cov_matrix[J] I_J = diag_matrix(rep_vector(1, J));
}

parameters {
  vector[J] alpha;
  matrix[J,K] beta;
  matrix[N,J] yy;
  cov_matrix[J] Omega_cov;
}

transformed parameters{
  cov_matrix[J] Marg_cov;
  Marg_cov = beta * beta'+ Omega_cov;
}

model {
  to_vector(beta) ~ normal(0, 1);
  to_vector(alpha) ~ normal(0, 10);
  Omega_cov ~ inv_wishart(J+6, I_J);
  for (n in 1:N) yy[n,] ~ multi_normal(alpha, Marg_cov);
  for (j in 1:J) DD[, j] ~ bernoulli_logit(yy[, j]);
}

generated quantities{
  matrix[J,J] betabeta =  beta * beta';
}



In [48]:
ps['alpha'].shape

(2000, 4, 6)

In [49]:
np.mean(ps['yy'][0],0).shape

(565, 6)

In [50]:
num_chains = 4
num_samples = ps['alpha'].shape[0]
num_warmup = ps['alpha'].shape[0]
num_iter = num_samples + num_warmup

In [51]:
for chain_number in range(num_chains):
    for i in range(num_samples):
        sign1 = np.sign(ps['beta'][i,chain_number,0,0])
        sign2 = np.sign(ps['beta'][i,chain_number,3,1])
        ps['beta'][i,chain_number,:3,0] = ps['beta'][i,chain_number,:3,0] * sign1
        ps['beta'][i,chain_number,3:,1] = ps['beta'][i,chain_number,3:,1] * sign2
        
        if 'Phi_cov' in ps.keys():
            ps['Phi_cov'][i,chain_number,0,1] = sign1 * sign2 * ps['Phi_cov'][i,chain_number,0,1]
            ps['Phi_cov'][i,chain_number,1,0] = ps['Phi_cov'][i,chain_number,0,1]


In [52]:
# nsim_N = 100
# data['K'] = 2
# PPP_vals, Dy, Dystr = get_PPP(data, ps, nsim_N)


In [53]:
%%opts Layout [fig_size=200]

chain_number = 1
plots = []
for j in range(data['J']):
    for k in range(data['K']):
        plots.append(plot_trace(ps['beta'][:,chain_number,j,k],
#              true_value=data['beta'][j,k],
             title = 'Posterior distribution for beta(%s,%s)'%(j,k)).\
                     options(fig_inches=8, aspect=3))
layout = hv.Layout(plots).options(show_title = True,
                                  vspace = .3,
                                  absolute_scaling=False,
                                  normalize=False) # use same y-range for all plots?

layout.cols(2)


:Layout
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_1_right_parenthesis :Curve   [x]   (y)

In [54]:
%%opts Layout [fig_size=200]

chain_number = 0

plots = []
for j in range(data['J']):
        plots.append(plot_trace(ps['alpha'][:,chain_number,j],
#              true_value=data['alpha'][j],
             title = 'Posterior distribution for alpha(%s)'%(j)).\
                     options(fig_inches=8, aspect=3))
layout = hv.Layout(plots).options(show_title = True,
                                  vspace = .3,
                                  absolute_scaling=False,
                                  normalize=False) # use same y-range for all plots?

layout.cols(2)


:Layout
   .Curve.Posterior_distribution_for_alpha_left_parenthesis_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_alpha_left_parenthesis_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_alpha_left_parenthesis_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_alpha_left_parenthesis_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_alpha_left_parenthesis_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_alpha_left_parenthesis_5_right_parenthesis :Curve   [x]   (y)

In [55]:
%%opts Layout [fig_size=200]

chain_number = 1

plots = []
for j in range(data['K']):
    for k in range(data['K']):
        plots.append(plot_trace(ps['Phi_cov'][:,chain_number,j,k],
#              true_value=data['Phi_cov'][j,k],
             title = 'Posterior distribution for Phi_cov(%s,%s)'%(j,k)).\
                     options(fig_inches=8, aspect=3))
layout = hv.Layout(plots).options(show_title = True,
                                  vspace = .3,
                                  absolute_scaling=False,
                                  normalize=True) # use same y-range for all plots?

layout.cols(2)


KeyError: 'Phi_cov'

In [56]:
nsim = 100
ppp_cn = np.empty(4)
for cn in range(4):
    PPP_vals, Dy, Dystr = get_PPP(data, ps, cn, nsim)
    
    ppp_cn[cn] = 100*np.sum(PPP_vals[:,0]<PPP_vals[:,1])/nsim
    print(ppp_cn[cn])
    
ppp_cn


37.0



30.0



41.0



41.0


array([37., 30., 41., 41.])